In [4]:
# 필요한 라이브러리 다시 가져오기
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# 데이터셋 로드

merged_df = pd.read_csv("D:/humanda/pjt-02/Mook/data-files/merged_df.csv")

# 'DATE' 열을 datetime 형식으로 변환
merged_df['DATE'] = pd.to_datetime(merged_df['DATE'])

# 'Month' 열 추가: 날짜 열에서 월 정보 추출
merged_df['Month'] = merged_df['DATE'].dt.month

# 재배 기간 정의
cultivation_periods = {
    "봄재배": {"planting": [3, 4, 5, 6], "harvesting": [6, 7]},       # 봄 재배의 파종 및 수확 기간
    "가을재배": {"planting": [7, 8, 9, 10], "harvesting": [10, 11]},  # 가을 재배의 파종 및 수확 기간
    "월동재배": {"planting": [9, 10, 11], "harvesting": [3, 4, 5]},   # 월동 재배의 파종 및 수확 기간
    "여름재배": {"planting": [5, 6, 7, 8], "harvesting": [9]}         # 여름 재배의 파종 및 수확 기간
}

# 각 재배 유형에 대해 파종 및 수확 기간을 나타내는 열 추가
for cultivation, periods in cultivation_periods.items():
    merged_df[cultivation] = (
        merged_df['Month'].isin(periods['planting'] + periods['harvesting'])  # 파종 및 수확 월에 해당하는지 여부
    ).astype(int)  # 결과를 0 또는 1로 변환

# '도매시장' 열에 대해 원-핫 인코딩 수행
# 각각의 도매시장 이름에 해당하는 새로운 열 생성 (값은 0 또는 1)
merged_df = pd.get_dummies(merged_df, columns=['도매시장'], prefix='Market')

# 피처(feature)와 타겟(target) 선택
features = [
    '총거래물량', 'weighted_price', '평균기온', '평균풍속', '평균강수량'  # 거래 및 기후 데이터
] + list(cultivation_periods.keys()) + [col for col in merged_df.columns if col.startswith('Market_')]  # 재배 유형 및 도매시장
X = merged_df[features]  # 입력 데이터(X) 설정
y = merged_df['평균가격']  # 출력 데이터(y) 설정

# 결측값 처리: 결측값이 포함된 행 제거
X = X.dropna()
y = y.loc[X.index]  # X에서 제외된 행에 맞춰 y도 정렬

# 피처 스케일링: 데이터 정규화를 위해 StandardScaler 사용
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터를 학습용(80%)과 테스트용(20%)으로 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 랜덤 포레스트의 하이퍼파라미터 그리드 정의
param_grid = {
    'n_estimators': [50, 100, 200],  # 트리의 개수
    'max_depth': [5, 10, 20, None],  # 트리의 최대 깊이
    'max_features': ['sqrt', 'log2', None]  # 사용되는 최대 피처 수
}

# 랜덤 포레스트 모델 초기화
rf_model = RandomForestRegressor(random_state=42)

# GridSearchCV 설정
grid_search = GridSearchCV(
    estimator=rf_model,    # 사용할 모델
    param_grid=param_grid,  # 하이퍼파라미터 그리드
    cv=5,  # 5-fold 교차 검증
    scoring='r2',  # R² 점수를 평가 지표로 사용
    verbose=2,  # 진행 상황 출력
    n_jobs=-1  # 가능한 모든 CPU 코어 사용
)

# 그리드 검색 실행
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터와 최적의 점수 확인
best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_params, best_score


Fitting 5 folds for each of 36 candidates, totalling 180 fits


({'max_depth': 20, 'max_features': None, 'n_estimators': 200},
 0.9379952433728882)

In [5]:
features

['총거래물량',
 'weighted_price',
 '평균기온',
 '평균풍속',
 '평균강수량',
 '봄재배',
 '가을재배',
 '월동재배',
 '여름재배',
 'Market_강릉도매시장',
 'Market_광주각화도매',
 'Market_광주서부도매',
 'Market_구리도매시장',
 'Market_구미도매시장',
 'Market_대구북부도매',
 'Market_대전노은도매',
 'Market_대전오정도매',
 'Market_부산반여도매',
 'Market_부산엄궁도매',
 'Market_서울가락도매',
 'Market_서울강서도매',
 'Market_수원도매시장',
 'Market_순천도매시장',
 'Market_안산도매시장',
 'Market_안양도매시장',
 'Market_울산도매시장',
 'Market_원주도매시장',
 'Market_인천구월도매',
 'Market_인천삼산도매',
 'Market_전주도매시장',
 'Market_정읍도매시장',
 'Market_진주도매시장',
 'Market_창원내서도매시장',
 'Market_창원팔용도매시장',
 'Market_천안도매시장',
 'Market_청주도매시장',
 'Market_춘천도매시장',
 'Market_충주도매시장',
 'Market_포항도매시장']

In [6]:
# Apply the best parameters found by GridSearchCV
optimized_rf_model = RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    max_features=best_params['max_features'],
    random_state=42
)

# Train the optimized model
optimized_rf_model.fit(X_train, y_train)

# Evaluate the model on training and test data
y_train_pred_optimized = optimized_rf_model.predict(X_train)
y_test_pred_optimized = optimized_rf_model.predict(X_test)

# Calculate performance metrics
train_mse_optimized = mean_squared_error(y_train, y_train_pred_optimized)
train_r2_optimized = r2_score(y_train, y_train_pred_optimized)
test_mse_optimized = mean_squared_error(y_test, y_test_pred_optimized)
test_r2_optimized = r2_score(y_test, y_test_pred_optimized)

{
    "Optimized Train MSE": train_mse_optimized,
    "Optimized Train R²": train_r2_optimized,
    "Optimized Test MSE": test_mse_optimized,
    "Optimized Test R²": test_r2_optimized
}


{'Optimized Train MSE': 66714.91316244028,
 'Optimized Train R²': 0.9929166542298278,
 'Optimized Test MSE': 334047.64375587914,
 'Optimized Test R²': 0.9629851202034397}